In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan
import numpy as np

In [3]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)
n_classes = Y.shape[1]
Y = np.argmax(Y, axis = 1)

100%|██████████| 102/102 [00:07<00:00, 14.27it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


In [7]:
imgs = X

In [8]:
batch_size = 256

In [9]:
if batch_size is None:
    batch_size = X.size(0)


In [10]:
# generate random samples
idxs_1 = np.random.randint(0, len(imgs), size=batch_size)
idxs_2 = np.random.randint(0, len(imgs), size=batch_size)

In [13]:
imgs_1 = imgs[idxs_1]
imgs_2 = imgs[idxs_2]

In [ ]:
z1 = apply_models(imgs_1, None, enc)
z2 = apply_models(imgs_2, None, enc)

In [ ]:
z = (z1 + z2) / 0.5

In [ ]:
if batch_size is None:
    batch_size = X.size(0)

# generate random samples
idxs_1 = np.random.randint(0, len(imgs), size=batch_size)
idxs_2 = np.random.randint(0, len(imgs), size=batch_size)
imgs_1 = imgs[idxs_1]
imgs_2 = imgs[idxs_2]
z1 = apply_models(imgs_1, None, enc)
z2 = apply_models(imgs_2, None, enc)
z = (z1 + z2) / 0.5
z = np_to_tensor(z)
x = np_to_tensor(imgs_1)